In [9]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [10]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [11]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)



In [12]:
# 2. Create a temporary view of the DataFrame.

home_sales.createOrReplaceTempView("home_sales")

spark.sql("""
SELECT * FROM home_sales
""").show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [13]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""
SELECT year(to_date(date,'yyyy-mm-dd')) AS year, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE bedrooms=4
GROUP BY year
ORDER BY year
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [14]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
SELECT date_built AS year, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [15]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT date_built AS year, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000 -- assuming the question is about the living space and not the lot footage
GROUP BY date_built
ORDER BY date_built
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [16]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view AS view, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE price>=350000
GROUP BY view
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- 1.0088093280792236 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales")

DataFrame[]

In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [19]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view AS view, ROUND(AVG(price),2) AS average_price
FROM home_sales
GROUP BY view
HAVING average_price>=350000

ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

--- 0.738494873046875 seconds ---


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales.write.parquet("parquet_home_sales", partitionBy="date_built", mode="overwrite")


In [21]:
# 11. Read the parquet formatted data.
parquet_home_sales_df = spark.read.parquet("parquet_home_sales")

In [22]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales_df.createOrReplaceTempView("parquet_home_sales")


In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT view AS view, ROUND(AVG(price),2) AS average_price
FROM parquet_home_sales
GROUP BY view
HAVING average_price>=350000

ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

--- 1.0276105403900146 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False